In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np

import sklearn
import sklearn.feature_extraction.text as sklearnText
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from collections import Counter

from functools import partial

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

import re
import string
import difflib

import pandas as pd
def multiplier(x):
    return x * 100

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# HIER EVEN JE CSV DIE JE GISTER MAAKTE EN OP DRIVE ZETTEN GOOIEN
test_data = pd.read_csv("Data/testsetlol.csv")
test_data.head()

,Unnamed: 0,Unnamed: 0.1,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,mean Q1,mean Q2
0,0,0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,48.175182,would trump presidency mean current internatio...,trump presidency affect students presently us ...,48.175182,0.161290,would trump presidency mean current internatio...,0.099746,0.168657,0.148489,0.140677
1,1,1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,66.666667,rockets look white,rockets boosters painted white,66.666667,0.400000,rockets look white!SPLIT!rockets boosters pain...,0.344642,0.411207,0.244901,0.290147
2,2,2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,52.000000,whats causing someone jealous,avoid jealous someone,52.000000,0.357143,whats causing someone jealous!SPLIT!avoid jeal...,0.380817,0.411207,0.219590,0.269113
3,3,3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,25.000000,much 30 kv hp,find conversion chart cc horsepower,25.000000,0.052632,much 30 kv hp!SPLIT!find conversion chart cc h...,0.000000,0.000000,0.165359,0.187500
4,4,4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,86.363636,best travel website spain,best travel website,86.363636,0.777778,best travel website spain!SPLIT!best travel we...,0.776515,0.776515,0.349145,0.306186


In [3]:
# HIER EVEN JE CSV DIE JE GISTER MAAKTE EN OP DRIVE ZETTEN GOOIEN
data_set = pd.read_csv("Data/datasettrainlol.csv")
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,...,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,mean Q1,mean Q2
0,0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,...,92.105263,step step guide invest share market india,step step guide invest share market,92.105263,1.000000,step step guide invest share market india!SPLI...,0.920307,0.895532,0.275582,0.267261
1,1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,...,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673,0.421053,story kohinoor kohinoor diamond!SPLIT!would ha...,0.424251,0.461171,0.184897,0.220267
2,2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,...,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000,0.300000,increase speed internet connection using vpn!S...,0.225765,0.225765,0.190308,0.173482
3,3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,...,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820,0.066667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000,0.157895,0.156969
4,4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",...,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101,0.263158,one dissolve water quikly sugar salt methane c...,0.272060,0.168368,0.222979,0.162995


In [4]:
# Scalen
data_set['jaccard'] = data_set['jaccard'].apply(multiplier)
data_set['cosine'] = data_set['cosine'].apply(multiplier)
data_set['cosineClean'] = data_set['cosineClean'].apply(multiplier)
test_data['jaccard'] = test_data['jaccard'].apply(multiplier)
test_data['cosine'] = test_data['cosine'].apply(multiplier)
test_data['cosineClean'] = test_data['cosineClean'].apply(multiplier)

# SITUATIES:
| **RF**|**Features Countvec**| **AC bekend**|
|--------|------|-----|
|50|100|0.78|
|100|100|0.79|
|50|500|0.80|
|75|250||
|75|400||
|75|500||
|75|650||
|75|800||
|75|1000||
|75|1250||
|75|1500||
|75|2000||
|75|5000||




In [5]:
def leukefunctienaam(COUNT, ESTIMATORS, FILENAME):
    # TEST
    vectorizer = CountVectorizer(max_features=COUNT)
    features = vectorizer.fit_transform(data_set.merged.values.astype('U'))
    X = vectorizer.transform(data_set.merged.values.astype('U'))
    names = vectorizer.get_feature_names()
    dense = X.todense()
    count_vect_df = pd.DataFrame(X.todense(), columns=names)

    # X_train
    data_set2 = data_set.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'id', 'question1',
           'question2', 'is_duplicate_x', 'is_duplicate_y', 'merged','cleanMerged',
             'clean1', 'clean2'], axis=1)
    X_train = pd.concat([data_set2, count_vect_df], axis=1)

    # X_test
    Y = vectorizer.transform(test_data.merged.values.astype('U'))
    count_vect_dfY = pd.DataFrame(Y.todense(), columns=names)
    test_data2 = test_data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'test_id', 'question1',
           'question2', 'merged','cleanMerged',
             'clean1', 'clean2'], axis=1)
    X_test = pd.concat([test_data2, count_vect_dfY], axis=1)

    # Y & fill
    y_train = data_set[['is_duplicate_y']]
    y_test = []
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    # RFM
    rfm = RandomForestClassifier(n_estimators=ESTIMATORS, oob_score=True, n_jobs=-1, random_state=None, max_features=None, min_samples_leaf=10)
    rfm.fit(X_train, y_train)
    y_pred=rfm.predict(X_train)
    print('accuracy train:', accuracy_score(y_pred, y_train))
    output=rfm.predict(X_test)
    print('output:', output)
    print('counter:', Counter(output))

    # DF
    submission = pd.concat([test_data.iloc[:, 2], pd.DataFrame(output)], axis=1, sort=False)
    submission.head()

    # CSV
    submission.to_csv(FILENAME, index = False)

# DEZE STAAT ER ALS TEST
Als je even checkt of deze runt (zou niet lang moeten duren), weet je dat die goed gaat

In [6]:
COUNT = 10
ESTIMATORS = 10 
FILENAME = 'result/submission-COUNT10-EST10.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

C:\Users\Marit\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


accuracy train: 0.8798348071436453
output: [0 0 0 ... 1 1 0]
counter: Counter({0: 49225, 1: 31901})


# Echte test

In [7]:
COUNT = 250
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT250-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

KeyboardInterrupt: 

In [ ]:
COUNT = 400
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT400-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 500
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT500-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 650
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT650-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 800
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT800-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 1000
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT1000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 1250
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT1250-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 1500
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT1500-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 2000
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT2000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 5000
ESTIMATORS = 75 
FILENAME = 'result/submission-COUNT5000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

# Extra tests
Ik weet niet hoe realistisch deze hieronder zijn, je moet maar even kijken of die gerunt zijn als je thuis komt. Mochten ze het doen, upload dan niet van laag naar hoog, maar beetje random wat omhoog gaat in verband met overfitten.

In [ ]:
COUNT = 5000
ESTIMATORS = 100
FILENAME = 'result/submission-COUNT5000-EST100.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 7500
ESTIMATORS = 75
FILENAME = 'result/submission-COUNT7500-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 10000
ESTIMATORS = 75
FILENAME = 'result/submission-COUNT10000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 12500
ESTIMATORS = 75
FILENAME = 'result/submission-COUNT12500-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 15000
ESTIMATORS = 75
FILENAME = 'result/submission-COUNT15000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)

In [ ]:
COUNT = 20000
ESTIMATORS = 75
FILENAME = 'result/submission-COUNT20000-EST75.csv'
leukefunctienaam(COUNT, ESTIMATORS, FILENAME)